In [1]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
import re
import os
import pytz
from datetime import datetime, timezone, timedelta, date
import datetime as dt

In [2]:
# Google Drive Setup
os.environ["GOOGLE_VALOREO_SOURCING_CREDENTIALS"]="../consol_inventario/valoreo_sourcing.json"
path_to_credentials_file = os.environ.get('GOOGLE_VALOREO_SOURCING_CREDENTIALS')
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = Credentials.from_service_account_file(path_to_credentials_file, scopes=scope)

In [3]:
full = pd.read_csv('../meli_api/inventario_meli_co.csv')
linio = pd.read_csv('../linio_api/linio_inv.csv')
homecenter = pd.read_csv('../homecenter_inv/homecenter_inv.csv')


In [4]:
dhl = pd.read_csv('dhl_inventory.csv')
dhl = dhl.pivot(index=['EAN_TEXT'], columns=['Estado'], values=['DHL']).reset_index()
dhl.columns = dhl.columns.to_flat_index()
cols = []
for column in dhl.columns:
    cols.append(' '.join(map(str,column)).strip())
dhl.columns = cols
dhl.rename(columns={'DHL Insumos': 'DHL Raw Materials', 'DHL Pend. Maquila': 'DHL Pending Maquila', 'DHL Repuestos': 'DHL Spare Parts', 'DHL Vendible': 'DHL Available'}, inplace=True)
dhl = dhl.groupby(['EAN_TEXT']).sum().reset_index()

In [5]:
full = full[['ean_text', 'available_qty', 'not_available_qty', 'total_qty']].groupby('ean_text').sum().reset_index()
full = full.rename(columns={'ean_text':'EAN_TEXT', 'available_qty': 'Meli_Full_Available', 'not_available_qty': 'Meli_Full_Unavailable', 'total_qty': 'Meli_Full_Total'})

In [6]:
linio = linio[['EAN_TEXT', 'FulfillmentBySellable']].groupby('EAN_TEXT').sum().reset_index()

In [7]:
linio = linio.rename(columns={'FulfillmentBySellable': 'Linio_Available'})

In [8]:
homecenter = homecenter[['EAN_TEXT', 'Inventario Total']].groupby('EAN_TEXT').sum().reset_index()
homecenter = homecenter.rename(columns={'Inventario Total': 'Homecenter Available'})

In [9]:
consolidated = dhl.merge(full, how='outer', on='EAN_TEXT')
consolidated = consolidated.merge(linio, how='outer', on='EAN_TEXT')
consolidated = consolidated.merge(homecenter, how='outer', on='EAN_TEXT')
consolidated.iloc[:,2:]=consolidated.iloc[:,2:].fillna(0)
consolidated = consolidated[['EAN_TEXT', 'Meli_Full_Available', 'Meli_Full_Unavailable', 'Meli_Full_Total', 'Linio_Available', 'Homecenter Available', 'DHL Available', 'DHL Pending Maquila', 'DHL Raw Materials', 'DHL Spare Parts']]


In [10]:
catalog = pd.read_csv('../buyplan_co/catalog_master_co.csv', dtype=str)

In [11]:
catalog = catalog[['EAN_TEXT', 'Brand', 'Description']]

In [12]:
catalog = catalog.rename(columns={'Description':'Item Name'})

In [13]:
catalog = catalog.dropna()

In [14]:
catalog = catalog.drop_duplicates(subset='EAN_TEXT')

In [15]:
consolidated = consolidated.merge(catalog, how='left', on='EAN_TEXT')

In [16]:
consolidated.to_csv('inventory.csv', index=False)
local_tz = pytz.timezone('America/Bogota')
current_day = datetime.today().astimezone(local_tz).strftime('%Y%m%d')
consolidated.to_csv(f'{current_day}_inventory.csv')